# My implementation of PPO Algorithm

## import some packages and check the env

In [1]:
import gymnasium as gym
import numpy as np
# these are new packages in this file
import panda_gym
import stable_baselines3
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env
from collections import deque
import matplotlib.pyplot as plt
import random
import imageio

import tensorboard
from torch.utils.tensorboard import SummaryWriter
%matplotlib inline

In [2]:
env_id = 'CartPole-v1'
env = gym.make(env_id)
state_space = env.observation_space
action_space = env.action_space.shape
print(f'state space: {state_space}')
print(f'action space: {action_space}')

state space: Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
action space: ()


## Build networks of PPO

The networks are same with A2C

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# we don't use the Categorical since the actions are continuous
# we should use the Normal to model the continous distribution
from torch.distributions import Categorical,Normal

In [4]:
writer = SummaryWriter()

In [5]:
# helper function: to concatenate the input and convert them to tensor
def t(x):
    if isinstance(x, np.ndarray):
            x = torch.tensor(x, dtype=torch.float)
    return x 

In [6]:
class Actor(nn.Module):
    def __init__(self, state_space, action_space,lr,device):
        super(Actor,self).__init__()
        self.model = nn.Sequential(
            nn.Linear(state_space, 64),
            nn.Tanh(),
            nn.Linear(64, 32),
            nn.Tanh(),
            nn.Linear(32, action_space), #the acition space is continous
            nn.Softmax()
        )
        self.optimizer = optim.Adam(self.parameters(),lr=lr)
        self.to(device)
    
    def forward(self, X):
        return self.model(t(X))

In [7]:
class Critic(nn.Module):
    def __init__(self, state_space,lr,device):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(state_space, 64),
            nn.ReLU(),
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
        self.optimizer = optim.Adam(self.parameters(),lr=lr)
        self.to(device)
    
    def forward(self, X):
        return self.model(t(X))

![clip](Algotithms/clipped.jpg)

In [8]:
class Agent():
    def __init__(
        self,
        env: gym.envs,
        state_space: int,
        action_space: int,
        lr: float,
        device: torch.device,
        gamma: float, # discounted rewards
        # gae_lambda: float,
        # policy_clip: float,
        # batch_size: int,
        timesteps_per_batch: int,
        n_updates_per_iteration: int,
        clip: float,
        n_training_episodes: int,
        n_eval_episodes: int,
        max_t: int 
    ):
         # init the variables
        self.env = env
        self.state_space = state_space
        self.action_space = action_space
        self.lr = lr
        self.device = device
        self.gamma = gamma
        # self.gae_lambda = gae_lambda
        # self.policy_clip = policy_clip
        # self.batch_size = batch_size
        self.timesteps_per_batch = timesteps_per_batch
        self.n_updates_per_iteration = n_updates_per_iteration
        self.clip = clip
        self.n_training_episodes = n_training_episodes
        self.n_eval_episodes = n_eval_episodes
        self.max_t = max_t
        
        # self.actor_critic = ActorCritic(state_space,action_space,
        #                                 lr,device)
        self.actor = Actor(state_space,action_space,lr,device)
        self.critic = Critic(state_space,lr,device)
   

        
    def act(self,state):
        probs = self.actor(state)
        dist = Categorical(probs.cpu())
        action = dist.sample()
        log_prob = dist.log_prob(action)
        # entropy = dist.entropy()
        return action.item(),log_prob
    
    def step(self,action):
        # the agent take the action and obtain the next observation and immedient reward
        # Then, push the infos into the buffer
        next_obs,reward,terminated,truncated,info = self.env.step(action)
        done = terminated or truncated
        return next_obs, reward, done
    
    # this is new in PPO
    def rollout(self):
        
        # batch_data
        batch_obs = []             # batch observations
        batch_acts = []            # batch actions
        batch_log_probs = []       # log probs of each action
        batch_rews = []            # batch rewards
        batch_rtgs = []            # batch rewards-to-go
        batch_lens = []            # episodic lengths in batch

        t = 0
        global step
        step = 0
        while t < self.timesteps_per_batch:
            ep_rews = []
            state,info = self.env.reset()
            done = False
            
            for ep_t in range(self.max_t):
                t += 1
                batch_obs.append(state)
                action,log_prob = self.act(state)
                state_,reward,done = self.step(action)
                ep_rews.append(reward)
                batch_acts.append(action)
                batch_log_probs.append(log_prob)
                
                if done:
                    break
                
                state = state_
                
            batch_lens.append(ep_t + 1)
            writer.add_scalar('ep_len',ep_t+1,step)
            batch_rews.append(ep_rews)
            writer.add_scalar('ep_avg_reward',np.mean(ep_rews),step)
            step += 1
            
        # Reshape data as tensors in the shape specified before returning
        batch_obs = torch.tensor(batch_obs, dtype=torch.float)
        batch_acts = torch.tensor(batch_acts, dtype=torch.float)
        batch_log_probs = torch.tensor(batch_log_probs, dtype=torch.float)
        
        # compute the reward to go
        for ep_rews in reversed(batch_rews):
            
            discounted_reward = 0
            for rew in reversed(ep_rews):
                discounted_reward = rew + discounted_reward * self.gamma
                batch_rtgs.insert(0,discounted_reward)
        
        batch_rtgs = torch.tensor(batch_rtgs,dtype=torch.float)
        return batch_obs, batch_acts, batch_log_probs, batch_rtgs, batch_lens

    
    
    
    
    
    def update(self):
        # compute the value,advantages and update the network
        batch_obs, batch_acts, batch_log_probs, batch_rtgs, batch_lens = self.rollout()
        ## compute the value
        V = self.critic(batch_obs)
        advantage = batch_rtgs - V.detach()
        # normalize advantage
        advantage = (advantage - advantage.mean()) / (advantage.std() + 1e-10)
        
        # compute the loss
        ## current V and action
        for _ in range(self.n_updates_per_iteration):
            logits = self.actor(batch_obs)
            dist = Categorical(logits)
            curr_log_probs = dist.log_prob(batch_acts)
            ## compute ratio
            ratios = torch.exp(curr_log_probs - batch_log_probs)
            ## compute surrogate loss
            surr1 = ratios * advantage
            surr2 = torch.clamp(ratios, 1 - self.clip, 1 + self.clip) * advantage
            
            # actor loss
            actor_loss = (-torch.min(surr1,surr2)).mean()
            self.actor.optimizer.zero_grad()
            actor_loss.backward(retain_graph=True)
            self.actor.optimizer.step()
            
            # critic loss
            V = self.critic(batch_obs)
            loss = nn.MSELoss()
            critic_loss = loss(V,batch_rtgs)
            self.critic.optimizer.zero_grad()
            critic_loss.backward()
            self.critic.optimizer.step()
            writer.add_scalar('policy loss',actor_loss)
            writer.add_scalar('critic loss',critic_loss)
            
            
        
        
    def train(self):
        for episode in range(self.n_training_episodes):
            self.update()
            if episode % 100 == 0:
                print(f'running {episode}') 
                
                
                

In [9]:
# Hyperparameter
state_space = env.observation_space.shape[0]
action_space = env.action_space.n
lr = 0.005
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gamma = 0.95
clip = 0.2
timesteps_per_batch = 4800
n_updates_per_iteration = 5
n_training_episodes = 1000
n_eval_episodes = 100
max_t = 1600

CartPoleAgent = Agent(
    env,
    state_space,
    action_space,
    lr,
    device,
    gamma,
    timesteps_per_batch,
    n_updates_per_iteration,
    clip,
    n_training_episodes,
    n_eval_episodes,
    max_t
)

In [10]:
CartPoleAgent.train()

/Users/gaohaitao/opt/anaconda3/envs/robotics/lib/python3.10/site-packages/torch/nn/modules/container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/var/folders/xt/fs8f0mrx5c7d2_ny5vdyy0ym0000gp/T/ipykernel_78359/383444071.py:99: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:248.)
  batch_obs = torch.tensor(batch_obs, dtype=torch.float)
/Users/gaohaitao/opt/anaconda3/envs/robotics/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([4801])) that is different to the input size (torch.Size([4801, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have th

running 0


/Users/gaohaitao/opt/anaconda3/envs/robotics/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([4809])) that is different to the input size (torch.Size([4809, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/gaohaitao/opt/anaconda3/envs/robotics/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([4810])) that is different to the input size (torch.Size([4810, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/gaohaitao/opt/anaconda3/envs/robotics/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([4822])) that is different to the input size (torch.Size([4822, 1])). This will likel

running 100


/Users/gaohaitao/opt/anaconda3/envs/robotics/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([4901])) that is different to the input size (torch.Size([4901, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/gaohaitao/opt/anaconda3/envs/robotics/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([4911])) that is different to the input size (torch.Size([4911, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/gaohaitao/opt/anaconda3/envs/robotics/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([4999])) that is different to the input size (torch.Size([4999, 1])). This will likel

...

/Users/gaohaitao/opt/anaconda3/envs/robotics/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([5193])) that is different to the input size (torch.Size([5193, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/gaohaitao/opt/anaconda3/envs/robotics/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([5187])) that is different to the input size (torch.Size([5187, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/gaohaitao/opt/anaconda3/envs/robotics/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([4887])) that is different to the input size (torch.Size([4887, 1])). This will likel

running 800


/Users/gaohaitao/opt/anaconda3/envs/robotics/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([5258])) that is different to the input size (torch.Size([5258, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/gaohaitao/opt/anaconda3/envs/robotics/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([4844])) that is different to the input size (torch.Size([4844, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/gaohaitao/opt/anaconda3/envs/robotics/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([4870])) that is different to the input size (torch.Size([4870, 1])). This will likel

running 900


/Users/gaohaitao/opt/anaconda3/envs/robotics/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([4994])) that is different to the input size (torch.Size([4994, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/gaohaitao/opt/anaconda3/envs/robotics/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([4971])) that is different to the input size (torch.Size([4971, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/gaohaitao/opt/anaconda3/envs/robotics/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([5277])) that is different to the input size (torch.Size([5277, 1])). This will likel

In [11]:
!tensorboard --logdir=runs

TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.14.1 at http://localhost:6006/ (Press CTRL+C to quit)
^C
